# Keras

# 1 Example

> ### a densely-connected network

```python
from keras.layers import Input, Dense
from keras.models import Model

# this model include a input layers and three dense layers
# return a tensor
inputs = Input(shape=(784,))

# a layer is callable on a tensor, and return a tensor
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)

predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# start training
model.fit(data, labels)
```

> ### Multi-input and multi-output models

![](./image/multi-input-multi-output-graph.png)

```python
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

main_input = Input(shape=(100,), dtype='int32', name='main_input')

x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)

lstm_out = LSTM(32)(x)

auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

auxiliary_input = Input(shape=(5,), name='aux_input')
x = keras.layers.concatenate([lstm_out, auxiliary_input])

x = Dense(64 activation='relu')(x)
x = Dense(64 activation='relu')(x)
x = Dense(64 activation='relu')(x)

main_output = Dense(1, activation='sigmoid', name='main_output')(x)

model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])

model.compile(optimizer='rmsprop', loss='binary_crossentropy', loss_weights=[1., 0.2])

model.fit([headline_data, additional_data], [labels, labels], epochs=50, batch_size=32)
# or
# model.compile(optimizer='rmsprop',
#               loss={'main_input': 'binary_crossentropy', 'aux_input': 'binary_crossentropy'},
#               loss_weights={'main_output': 1., 'aux_output': 0.2})

# model.fit({'main_input': headline_data, 'aux_input': additional_data},
#           {'main_output': labels, 'aux_output': labels},
#           epochs=50, batch_size=32)
```

> ### Shared layers

```python
import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model

tweet_a = Input(shape=(280, 256))
tweet_b = Input(shape=(280, 256))

shared_lstm = LSTM(64)

encoded_a = shared_lstm(tweet_a)
encoded_b = shared_lstm(tweet_b)

merged_vector = keras.layers.concatenate([encoded_a, encoded_b])

predictions = Dense(1, activation='sigmoid')(merged_vector)

model = Model(inputs=[tweet_a, tweet_b], outputs=predictions)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit([data_a, data_b], labels, epochs=10)
```

> ### The concept of layer 'node'

input_shape, get_inout_shape_at(), output, get_output_at()

```python
a = Input(shape=(280, 256))

lstm = LSTM(32)
encoded_a = lstm(a)

assert lstm.output == encoded_a
```
if the layer has multiple inputs:
```python
a = Input(shape=(280, 256))
b = Input(shape=(280, 256))

lstm = LSTM(32)
encoded_a = lstm(a)
encoded_b = lstm(b)

assert lstm.get_output_at(0) == encoded_a
assert lstm.get_output_at(1) == encoded_b
```
input_shape
```python
a = Input(shape=(32, 32, 3))
b = Input(shape=(64, 64, 3))

conv = Conv2D(16, (3, 3), padding='same')
conved_a = conv(a)

# Only one input so far, the following will work:
assert conv.input_shape == (None, 32, 32, 3)

conved_b = conv(b)
# now the `.input_shape` property wouldn't work, but this does:
assert conv.get_input_shape_at(0) == (None, 32, 32, 3)
assert conv.get_input_shape_at(1) == (None, 64, 64, 3)
```

> ### Inception module

```python
from keras.layers import Conv2D, MaxPooling2D, Input

input_img = Input(shape(256, 256, 3))

tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_img)
tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3)

output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=1)
```

> ### Residual connection on a convolution layer

```python
from keras.layers import Conv2D, Input

x = Input(shape=(256, 256, 3))

y = Conv2D(3, (3, 3), padding='same')(x)

z = keras.layers.add([x, y])
```

> ### Shared vision model

可以理解为一个模型由多个模型组成

```python
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten
from keras.models import Model

digit_input = Input(shape=(27, 27, 1))
x = Conv2D(64, (3, 3))(digit_input)
x = Conv2D(64, (3, 3))(x)
x = MaxPooling2D((2, 2))(x)

out = Flatten()(x)

vision_model = Model(digit_input, out)

digit_a = Input(shape=(27, 27, 1))
digit_b = Input(shape=(27, 27, 1))

out_a = vision_model(digit_a)
out_b = vision_model(digit_b)

concatenated = keras.layers.concatenate([out_a, out_b])
out = Dense(1, activation='sigmoid')(concatenated)

classification_model = Model([digit_a, digit_b], out)
```

> ### Visual question answering model

```python
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model, Sequential

vision_model = Sequential()
vision_model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
vision_model.add(Conv2D(64, (3, 3), activation='relu'))# 222
vision_model.add(MaxPooling2D((2, 2)))# 111

vision_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))#111
vision_model.add(Conv2D(128, (3, 3), activation='relu'))#109
vision_model.add(MaxPooling2D((2, 2)))#54

vision_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))#54
vision_model.add(Conv2D(256, (3, 3), activation='relu'))#52
vision_model.add(MaxPooling2D((2, 2)))#26

vision_model.add(Flatten())

image_input = Input(shape=(224, 224, 3))
encoded_image = vision_model(image_input)

question_input = Input(shape=(100,), dtype='int32')
embedded_question = Embedding(input_dim=10000, output_dim=256, input_length=100)(question_input)
encoded_question = LSTM(256)(embedded_question)

merged = keras.layers.concatenate([encoded_question, encoded_image])

output = Dense(1000, activation='softmax')(merged)

vqa_model = Model(inputs=[image_input, question_input], outputs=output)
```

> ### Video question answering model

```python
from keras.layers import TimeDistributed

video_input = Input(shape=(100, 224, 224, 3))

encoded_frame_sequence = TimeDistributed(vision_model)(video_input)
encoded_video = LSTM(256)(encoded_frame_sequence)

question_encoder = Model(inputs=question_input, outputs=encoded_question)

video_question_input = Input(shape=(100,), dtype='int32')
encoded_video_question = question_encoder(video_question_input)

merged = keras.layers.concatenate([encoded_video, encoded_video_question])

output = Dense(1000, activation='softmax')(merged)

video_qa_model = Model(inputs=[video_input, video_question_input], outputs=output)
```

<hr style='border:none;border-top:3px solid red'/>

# 2 models

- `model.layers`is a flattened list of the layers comprising the model.
- `model.input`is the list of input tensors of the model
- `model.outputs`
- `model.summary()`prints a summary representation
- `model.get_config()`return a dictionary containing the configuration of the model.
    - `Model.from_config()`
    - `Sequential.from_config()`
- `model.get_weights()`return a list of all weight tensors in the model.
- `model.set_weights(weights)`sets the values of the weights of the model.
- `model.to_json()`return a representation of the model as a json string.only the architecture.
    - `keras.models.model_from_json()`
- `model.to_yaml()`return a representation of the model as a yaml string ,only architecture.
    - `keras.models.model_from_yaml()`
- `model.save_weights(filepath)`save the weights of the model as a HDF5 file.
    - `model.load_weights(filepath, by_name=False)`

<hr style='border:none;border-top:3px solid red;'>

# 3 Sequential

```python
compile(self, optimizer, loss=None, metrics=None)

hist = fit(self, x=None, y=None, batch_size=None, epochs=1, validation_data=None, shuffle-True)
hist.history

evaluate(self, x=None, y=None, batch_size=None, sample_weight=None, **kwargs)

pred = predict(self, x, batch_size=None, verbose=0, steps=None)

loss = train_on_batch(self, x, y, sample_weight=None, class_weight=None)

loss = test_on_batch(self, ...)

pred = predict_on_batch(self, x)

hist = fit_generator(self, generator, ...)

loss = evaluate_generator(self, generator, ...)

pred = predict_generator(self, generator, ...)

layer = get_layer(self, name=None, index=None)
```

<hr style='border:none;border-top:3px solid red;'>

# 4 layers

- `layer.get_weights()`
- `layer.set_weights()`
- `layer.get_config()`
- `layer.input`
- `layer.output`
- `layer.input_shape`
- `layer.output_shape`
- `layer.get_input_at()`
- `layer.get_output_at()`
- `layer.get_input_shape_at()`
- `layer.get_output_shape_at()`

<hr style='border:none;border-top:3px solid red;'>

# 5 CoreLayers

**Dense**
```python
keras.layers.Dense(units, activation)
```

**Activation**
```python
keras.layers.Activation(activation)
```

**Dropout**
```python
keras.layers.Dropout(rate, noise_shape=None, seed=None)
```

**Flatten**
```python
keras.layers.Flatten(data_format=None)
```

**Input**
```python
keras.engine.input_layer.Input()
```

**Reshape**
```python
# Reshape an output to a certain shape
keras.layers.Reshape(target_shape)
```



In [59]:
keras.engine.input_layer.Input?

In [60]:
keras.layers.Input?